## 교차검증

### 데이터불러오기 : 와인data

In [1]:
import pandas as pd

In [2]:
wine = pd.read_csv('./data/08_wine.csv')
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [3]:
# 독립/종속변수 분리
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

In [4]:
print(data.shape,target.shape)

(6497, 3) (6497,)


### 교차검증 : train_test_split() 방식

In [5]:
### 훈련데이터 : 검증데이터 : 테스트데이터로 쪼개기
# - 보통 6 : 2 : 2
# - 보통 많이 사용하는 방식

In [6]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(data, target,
                                                                     test_size = 0.2, random_state=42)

print(train_input.shape)
print(train_target.shape)
print(test_input.shape)
print(test_target.shape)

(5197, 3)
(5197,)
(1300, 3)
(1300,)


In [7]:
### 2. 훈련데이터를 이용해 검증데이터 생성
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target,
                                                                     test_size = 0.2, random_state=42)

print(sub_input.shape)
print(sub_target.shape)
print(val_input.shape)
print(val_target.shape)

(4157, 3)
(4157,)
(1040, 3)
(1040,)


In [8]:
### 사용하는 클래스 : DecisionTreeClassifier
### 사용하는 패키지 : sklearn.tree

from sklearn.tree import DecisionTreeClassifier

### 클래스 생성
# - 실제 사용시에는 random_state은 사용하지 않는 것이 좋음
# - random_state의 값이 변경되면 정확도 점수 변경
# - 보통 random_state은 실전에서는 사용하지 않는 편
dtc = DecisionTreeClassifier(random_state=42)

dtc.fit(sub_input, sub_target)

print(dtc.score(sub_input, sub_target))
print(dtc.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [9]:
print(dtc.score(test_input, test_target))

0.8569230769230769


##  교차검증 시작

In [10]:
### 훈련데이터와 검증데이터를 구분할 필요없이
# - 내부적으로 구분해서 사용
# - 내부적으로 구분해 사용하는 데이터는 훈련(train)데이터
# - 데이터 준비는 기존처럼 훈련과 테스트데이터만 준민
# - 검증데이터는 별도로 준비 안해도 됨

In [11]:
### 교차검증에 사용되는 주요 키워도
# 3-폴드(fold) 교차검증
#  : 훈려데이터를 세부분으로 나눠서 데이터를 쪼개고 수행
#  : 구간마다 훈련데이터를 검증데이터로 바꿔가면서 수행
#  : k-폴드 교차검증 또는 k-겹 교차검증이라고 한다

# 보통 5-폴드 교차검증 또는 10-폴드 교차검증 사용
#  : 훈련데이터로 80% 이상을 모두 모델 훈련에 사용 가능

In [12]:
# 사용하는 데이터 다시 확인
print(train_input.shape, train_target.shape)
print(test_input.shape, test_target.shape)

(5197, 3) (5197,)
(1300, 3) (1300,)


In [13]:
###  교차검증 모듈 : cross_validate
from sklearn.model_selection import cross_validate

# dtc : 결정트리 훈련모델
# 두번째값 : 훈련데이터(fold에서 훈련데이터 쪼갤때 사용)
# 세번째값 : 검증데이터(fold에서 검증데이터 쪼갤때 사용)
scores = cross_validate(dtc, train_input, train_target)
print(scores)
print('======================')

### 결과값
# test_score : 최종검증평균값

# test_score의 평균값이 최종 훈련모델의 평가점수(정확도)
import numpy as np
print("최종 score = ", np.mean(scores['test_score']))

{'fit_time': array([0.00598431, 0.00625324, 0.0064528 , 0.00701785, 0.00596285]), 'score_time': array([0.        , 0.00072742, 0.00051284, 0.        , 0.        ]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}
최종 score =  0.855300214703487


In [14]:
### 훈련데이터를 섞거나, 폴드의 갯수를 지정할 수 있는 클래스
# - StratifiedKFold
from sklearn.model_selection import StratifiedKFold

# 교차검증 함수 그대로 사용
# cv : 분할기 속성
#    : 분할기로 StratifiedKFold 클래스 사용
#    : 속성값이 없을 경우 기본 fold는 5, 기본 섞지는 않음
scores = cross_validate(dtc, train_input, train_target,
                       cv = StratifiedKFold())
print(scores)
print('======================')

# test_score의 평균값이 최종 훈련모델의 평가점수(정확도)
import numpy as np
print("최종 score = ", np.mean(scores['test_score']))

{'fit_time': array([0.00598502, 0.00520468, 0.00549054, 0.006042  , 0.0059526 ]), 'score_time': array([0.        , 0.00049567, 0.00049114, 0.00097132, 0.        ]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}
최종 score =  0.855300214703487


In [15]:
### 섞기

### 훈련데이터를 섞거나, 폴드의 갯수를 지정할 수 있는 클래스
# - StratifiedKFold
from sklearn.model_selection import StratifiedKFold

# 교차검증 함수 그대로 사용
# cv : 분할기 속성
#    : 분할기로 StratifiedKFold 클래스 사용
#    : 속성값이 없을 경우 기본 fold는 5, 기본 섞지는 않음

# n_splits : fold 갯수, shuffle : 섞기
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dtc, train_input, train_target,
                       cv = splitter )
print(scores)
print('======================')

# test_score의 평균값이 최종 훈련모델의 평가점수(정확도)
import numpy as np
print("최종 score = ", np.mean(scores['test_score']))

{'fit_time': array([0.00698233, 0.00661111, 0.00598335, 0.00784492, 0.0069809 ,
       0.00594401, 0.00694084, 0.00698185, 0.00601006, 0.0059762 ]), 'score_time': array([0.        , 0.0003705 , 0.00099826, 0.00013232, 0.0009973 ,
       0.00103879, 0.        , 0.00099659, 0.        , 0.00099349]), 'test_score': array([0.83461538, 0.87884615, 0.85384615, 0.85384615, 0.84615385,
       0.87307692, 0.85961538, 0.85549133, 0.85163776, 0.86705202])}
최종 score =  0.8574181117533719


## 하이퍼파라미터 튜닝(AutoML)

In [16]:
### 교차검증, 하이퍼파라미터 찾기, 모델훈련을 한번에 자동으로 수행
# - 이런 개념을 AutoML이라 함
# - 사용패키지 : sklearn.model_selection
# - 사용클래스 : GridSearchCV
# - 그리드 서치라고도 함

## 결정트리의 max_depth(트리의 깊이) 값 찾기
##### - 그리드서치(GridSearchCV)  객체(모델) 사용

In [17]:
from sklearn.model_selection import GridSearchCV

# 찾을 하이퍼파라미터는 딕셔너리로
# 실제 파라미터 변수이름 그대로, 찾을 범위 지정
params = {'max_depth' : range(5, 20, 1)}

# 객체(모델) 생성
# - 첫번째 값 : 훈련모델 넣기
# - 두번째 값 : 찾을 하이퍼파라미터 값들(딕셔너리로 정의)
# - 세번째 값 : CPU 코어 갯수(-1은 모든 코어 사용, 벙렬 처리)
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),
                 params, n_jobs = -1)

# 훈련시키기
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20)})

In [18]:
# 그리드서치의 결과
# - 하이퍼파라미터의 찾은 값 알려줌
# - 중요 : 훈련모델이 생성되어 반환됨
print(gs.best_estimator_)

DecisionTreeClassifier(max_depth=8, random_state=42)


In [19]:
### 그리드서치를 통해 생성된 훈련모델 통해
# 평가해보기, 훈련데이터 전체 사용

dtc = gs.best_estimator_
print(dtc.score(train_input, train_target))

0.9003271117952665


In [20]:
print(gs.best_params_)
print(gs.cv_results_)

{'max_depth': 8}
{'mean_fit_time': array([0.00651031, 0.01274037, 0.00913625, 0.00978737, 0.01015539,
       0.01652818, 0.00898504, 0.00796976, 0.00822134, 0.00965009,
       0.00909348, 0.00833569, 0.00914993, 0.00925379, 0.00824676]), 'std_fit_time': array([0.00069228, 0.00573343, 0.00192785, 0.00241792, 0.0038075 ,
       0.01459352, 0.00064907, 0.0008708 , 0.00195886, 0.00152116,
       0.00082569, 0.00043632, 0.00193921, 0.00209801, 0.00096957]), 'mean_score_time': array([0.00182185, 0.00161777, 0.00117211, 0.00141177, 0.00076838,
       0.00121403, 0.00099769, 0.00120139, 0.00099573, 0.00062289,
       0.00079155, 0.0006424 , 0.00097899, 0.00090585, 0.00119333]), 'std_score_time': array([9.56632658e-04, 8.01437712e-04, 4.13755573e-04, 6.75181558e-04,
       3.92362505e-04, 4.06220819e-04, 1.09361123e-06, 3.95545031e-04,
       1.17920586e-05, 3.88375174e-04, 3.95799458e-04, 5.30954217e-04,
       9.02343486e-05, 1.23616996e-04, 7.43538488e-04]), 'param_max_depth': masked_array(d

In [21]:
print(gs.cv_results_.keys())

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_max_depth', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])


In [22]:
### params 값 확인하기
print(gs.cv_results_['params'])

[{'max_depth': 5}, {'max_depth': 6}, {'max_depth': 7}, {'max_depth': 8}, {'max_depth': 9}, {'max_depth': 10}, {'max_depth': 11}, {'max_depth': 12}, {'max_depth': 13}, {'max_depth': 14}, {'max_depth': 15}, {'max_depth': 16}, {'max_depth': 17}, {'max_depth': 18}, {'max_depth': 19}]


In [23]:
# 첫번째 fold 값
print(gs.cv_results_['split0_test_score'])

[0.84711538 0.84807692 0.85769231 0.85288462 0.85769231 0.84519231
 0.85865385 0.86730769 0.86538462 0.86826923 0.86730769 0.86442308
 0.86346154 0.86634615 0.87211538]


In [24]:
### 훈련데이터 최종검증하기
dtc.score(test_input, test_target)

0.8584615384615385

In [25]:
gs.cv_results_['split0_test_score']

array([0.84711538, 0.84807692, 0.85769231, 0.85288462, 0.85769231,
       0.84519231, 0.85865385, 0.86730769, 0.86538462, 0.86826923,
       0.86730769, 0.86442308, 0.86346154, 0.86634615, 0.87211538])

In [26]:
df = pd.DataFrame(gs.cv_results_['split0_test_score'])
# columns = ['max_depth5', 'max_depth6', 'max_depth7','max_depth8', 'max_depth9', 'max_depth10', 'max_depth11', 
#            'max_depth12', 'max_depth13', 'max_depth14','max_depth15',
#            'max_depth16', 'max_depth17', 'max_depth18', 'max_depth19']

In [27]:
a = []
for i in range(0,5):
    df = gs.cv_results_[f'split{i}_test_score']
    a.append(df)
    
df = pd.DataFrame(a)

df.columns = ['max_depth5', 'max_depth6', 'max_depth7','max_depth8', 'max_depth9', 'max_depth10', 'max_depth11', 
           'max_depth12', 'max_depth13', 'max_depth14','max_depth15',
           'max_depth16', 'max_depth17', 'max_depth18', 'max_depth19']
df

,max_depth5,max_depth6,max_depth7,max_depth8,max_depth9,max_depth10,max_depth11,max_depth12,max_depth13,max_depth14,max_depth15,max_depth16,max_depth17,max_depth18,max_depth19
0,0.847115,0.848077,0.857692,0.852885,0.857692,0.845192,0.858654,0.867308,0.865385,0.868269,0.867308,0.864423,0.863462,0.866346,0.872115
1,0.863462,0.854808,0.851923,0.854808,0.859615,0.847115,0.850962,0.848077,0.853846,0.851923,0.853846,0.849038,0.850000,0.853846,0.856731
2,0.880654,0.873917,0.875842,0.878730,0.871030,0.876805,0.880654,0.882579,0.873917,0.870067,0.868142,0.876805,0.871992,0.873917,0.878730
3,0.836381,0.843118,0.843118,0.846968,0.842156,0.846968,0.852743,0.848893,0.853705,0.858518,0.854668,0.859480,0.852743,0.856593,0.849856
4,0.861405,0.859480,0.847931,0.865255,0.854668,0.852743,0.848893,0.848893,0.839269,0.841193,0.837344,0.838306,0.839269,0.838306,0.837344


## 찾을 하이퍼파라미터 여러개 사용하기

In [28]:
# min_impurity_devrease : 결정트리 노드 분리시 불순도 감소 최소량
# max_depth : 결정트리 깊이
# min_samples_split : 결정트리 노드를 나누기 위한 최소 샘플 갯수 
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001),
          'max_depth' : range(5, 20, 1),
          'min_samples_split' : range(2, 100, 10)}

# ### 그리드서치 훈련시키기
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42),
                 params, n_jobs = -1)

gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [29]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [30]:
print(gs.best_estimator_)

DecisionTreeClassifier(max_depth=14, min_impurity_decrease=0.0004,
                       min_samples_split=12, random_state=42)


In [31]:
dtc = gs.best_estimator_
print(dtc.score(train_input, train_target))
print(dtc.score(test_input, test_target))

0.892053107562055
0.8615384615384616


In [32]:
### 10-폴드 지정하기
splitter = StratifiedKFold(n_splits=7, shuffle=True, random_state=42)

params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001),
          'max_depth' : range(5, 20, 1),
          'min_samples_split' : range(2, 100, 10)}

# ### 그리드서치 훈련시키기
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42),
                 params, cv = splitter, n_jobs = -1)

gs.fit(train_input, train_target)

GridSearchCV(cv=StratifiedKFold(n_splits=7, random_state=42, shuffle=True),
             estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [33]:
dtc = gs.best_estimator_
print(dtc.score(train_input, train_target))
print(dtc.score(test_input, test_target))

0.9230325187608236
0.8692307692307693


## 랜덤서치(Random Search)

In [34]:
### 랜덤서치 사용 시 아래 조건 만족
# - 하이퍼파라미터값이 수치인 것들만 사용
# - 범위나 간격을 미리 정하기 어려울 경우
# - 너무 많은 매개변수가 있어서 그리드서치 수행시간이 오래 걸리는 경우
# - 매개변수를 샘플링할 때 확률분포객체를 전달

# 사용 모듈 : uniform(실수값), randint(정수값)

In [35]:
from scipy.stats import uniform, randint

In [36]:
### 범위 설정
rgen = randint(0, 10)
rgen

### 범위 내에서 임의값 출력(확률적으로 고르게 뽑아냄)
print(rgen.rvs(100))

[7 1 4 3 3 0 0 1 7 7 7 4 1 9 9 8 7 5 6 7 5 3 1 6 6 3 4 4 4 4 0 0 8 0 1 4 5
 8 4 3 5 3 7 7 2 0 1 5 8 0 9 9 6 0 4 6 7 0 6 6 8 2 0 8 4 1 0 9 1 8 8 4 3 7
 3 1 6 7 5 6 1 7 8 6 1 1 6 6 7 9 0 8 5 6 7 3 9 8 9 8]


In [37]:
# 실수값
# 범위 설정
ugen = uniform(0, 1)

# 임의값 50개 조회
print(ugen.rvs(50))

[0.57009841 0.76952276 0.45310644 0.10347861 0.37213556 0.19192023
 0.52721209 0.74768905 0.45284391 0.74091816 0.62128918 0.29896951
 0.03135548 0.24054547 0.08306477 0.00107187 0.26929046 0.28310332
 0.54777087 0.82531098 0.41708955 0.81991761 0.57152311 0.65714878
 0.89675778 0.50674736 0.20449764 0.15297905 0.14505624 0.51398907
 0.42708031 0.3390784  0.96817428 0.5710254  0.80258178 0.2982642
 0.94971437 0.61771167 0.09403351 0.47581476 0.95301219 0.28683603
 0.13583562 0.62434866 0.19276739 0.53745879 0.29746464 0.87581313
 0.8558492  0.11128178]


In [38]:
### 하이퍼파라미터 찾을 매개변수 정의하기
params = {'min_impurity_decrease' : uniform(0.0001, 0.001),
          'max_depth' : randint(20, 50),
          'min_samples_split' : randint(2, 25)}

# 랜덤서치 클래스(모델) : RandomizedSearchCV()
# 사용방법은 그리드서치와 동일
# 결정트리 훈련 모델 사용, 훈련반복 횟수 : 100, 코어 전체. 시드 42

from sklearn.model_selection import RandomizedSearchCV

rs = RandomizedSearchCV(DecisionTreeClassifier(random_state = 42),
                 params, n_iter = 100, n_jobs = -1, random_state = 42)

rs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x00000184E38D00D0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x00000184E38D3100>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x00000184E38D2410>},
                   random_state=42)

In [39]:
print(rs.cv_results_.keys())

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_max_depth', 'param_min_impurity_decrease', 'param_min_samples_split', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])


In [40]:
rs.cv_results_['mean_test_score']

array([0.86203543, 0.86203524, 0.86453672, 0.8639598 , 0.86434441,
       0.86184534, 0.86280669, 0.86684608, 0.86607592, 0.86203561,
       0.85972662, 0.8603054 , 0.86338306, 0.86145813, 0.86222792,
       0.86145739, 0.86761605, 0.86261272, 0.86184238, 0.86396109,
       0.86453709, 0.86646165, 0.86184312, 0.86665322, 0.86165044,
       0.86030484, 0.86299845, 0.86742356, 0.86203543, 0.86838658,
       0.86319186, 0.86819334, 0.85799659, 0.86242023, 0.86203561,
       0.86299752, 0.86261309, 0.86049733, 0.86857833, 0.86299752,
       0.86800085, 0.86203543, 0.8664635 , 0.86415451, 0.86588473,
       0.86338362, 0.8678091 , 0.86357611, 0.8624206 , 0.85857315,
       0.86203543, 0.86742374, 0.86607629, 0.86838639, 0.86107333,
       0.86030466, 0.86203543, 0.86376805, 0.86396146, 0.86954098,
       0.86954079, 0.86800104, 0.86145739, 0.86242023, 0.85972662,
       0.86261327, 0.86684626, 0.86703913, 0.86472958, 0.86280503,
       0.86607814, 0.86530632, 0.86800104, 0.86203543, 0.86088

In [41]:
### 훈련 100회 중 가장 큰값이 최종 평가 점수
np.max(rs.cv_results_['mean_test_score'])

0.8695409787517583

In [42]:
dtc = rs.best_estimator_
print(dtc)
print('==========================')
print(rs.best_params_)
print('==========================')
print(dtc.score(train_input, train_target))
print(dtc.score(test_input, test_target))

DecisionTreeClassifier(max_depth=29, min_impurity_decrease=0.000437615171403628,
                       min_samples_split=16, random_state=42)
{'max_depth': 29, 'min_impurity_decrease': 0.000437615171403628, 'min_samples_split': 16}
0.8895516644217818
0.8623076923076923
